# IMDb database cleaning

Two databases are at use
- title.basics contains the basic information about each title, runtime length, year of relise, and genres
- title.ratings contains the aggregated ratings for each title

As these datasets weigh over 1GB, they are not uploaded to the GitHub repo, instead you should download them to the same folder from [here](https://datasets.imdbws.com/)

In [25]:
import pandas as pd

In [2]:
tb = pd.read_csv("title.basics.tsv", delimiter="\t" )

/var/folders/cj/1q3301cj5435wzpyw_xn5nhr0000gp/T/ipykernel_44706/765276917.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  tb = pd.read_csv("title.basics.tsv", delimiter="\t" )


In [3]:
tb.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
tb.describe()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
count,10854461,10854461,10854443,10854443,10854461,10854461,10854461,10854461,10854250
unique,10854461,11,4802488,4825937,30,153,97,908,2372
top,tt9916880,tvEpisode,Episode #1.1,Episode #1.1,0,\N,\N,\N,Drama
freq,1,8266441,52084,52084,10448563,1404650,10729125,7422878,1232695


As we can see, there are nearly 11 million rows, 8 mil out of which describe "tvEpisode", which are episodes for "tvSeries" entries, are redundant and do not contain any additional value.

In [5]:
tb["titleType"].value_counts()

titleType
tvEpisode       8266441
short           1017733
movie            709615
video            293250
tvSeries         267188
tvMovie          148433
tvMiniSeries      55104
tvSpecial         47696
videoGame         38521
tvShort           10479
tvPilot               1
Name: count, dtype: int64

It makes sense to leave out tvEpisodes, videoGames and videos from our future anaylysis, as they are redundant or irrelevant. We will leave out the only tvPilot as well, to reduce dataset complexity without decreasing its quality.

In [6]:
irrelevant_title_types = ["videoGame", "tvEpisode", "tvPilot", "video"]
filtered_tb = tb[~tb['titleType'].isin(irrelevant_title_types)]
filtered_tb.describe()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
count,2256248,2256248,2256244,2256244,2256248,2256248,2256248,2256248,2256248
unique,2256248,7,1687447,1710151,4,152,97,782,2302
top,tt9916856,short,Home,Home,0,\N,\N,\N,"Drama,Short"
freq,1,1017733,487,466,2230324,166076,2130912,880876,200965


In [7]:
filtered_tb["titleType"].value_counts()


titleType
short           1017733
movie            709615
tvSeries         267188
tvMovie          148433
tvMiniSeries      55104
tvSpecial         47696
tvShort           10479
Name: count, dtype: int64

In [8]:
del tb

After removing unnecessary data from our original dataset, we need to handle null values, which are denoted as "\N". 

In [9]:
filtered_tb = filtered_tb.replace("\\N", np.nan)
filtered_tb.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


We can immideately see the difference, as endYear shrinks almost 20 times, and \N stops being its most frequent value

In [10]:
filtered_tb.describe()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
count,2256248,2256248,2256244,2256244,2256248,2090172,125336,1375372,2138870
unique,2256248,7,1687447,1710151,4,151,96,781,2301
top,tt9916856,short,Home,Home,0,2016,2019,10,"Drama,Short"
freq,1,1017733,487,466,2230324,98422,6999,47940,200965


Now we import title.ratings, which contain ratings and vote numbers that we can join to filtered_tb

In [11]:
ratings = pd.read_csv("title.ratings.tsv", sep="\t")
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2041
1,tt0000002,5.7,272
2,tt0000003,6.5,1994
3,tt0000004,5.4,178
4,tt0000005,6.2,2753


In [20]:
final_tb = filtered_tb.merge(ratings, on="tconst", how="left")
final_tb.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short",5.7,2041.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short",5.7,272.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance",6.5,1994.0
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short",5.4,178.0
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short",6.2,2753.0


The populated and cleaned database now can be used for analyses later

In [24]:
final_tb.to_csv("cleaned-imdb-dataset.csv")
del filtered_tb
del final_tb